# Playground for Multihead Attention

In [63]:
# Run previous notebook to have data prepared
%run "./DataPreparation.ipynb"

## Prototype Attention Head
Implementation of a single attention head

In [64]:
import torch
import torch.nn as nn

class CausalSelfAttention(nn.Module):
    
    def __init__(self, input_dim, output_dim, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.output_dim = output_dim
        
        # init weight matrices for input -> query, key, weight projection
        self.W_query = nn.Linear(input_dim, output_dim, bias=qkv_bias)
        self.W_key = nn.Linear(input_dim, output_dim, bias=qkv_bias)
        self.W_value = nn.Linear(input_dim, output_dim, bias=qkv_bias)
        
        # init dropout (prevent overfitting)
        self.dropout = nn.Dropout(dropout) # dropout is a probability
        
        # registers static causal masking matrix (diagonal) in same device as model (is self.causal_mask)
        self.register_buffer('causal_mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # Causal Masking on diagonal triangle

    def forward(self, x):
        
        # stores dimensions of input (here only context length needed)
        batch_size, con_len, in_dim = x.shape # set them to the shape of x ( 8, 4, 256 )
        
        # using the weight matrices for projection of the input to query, key and value
        # broadcasting (for badges)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # calculates all attention scores
        # Compares query for every input token to all keys
        attn_scores = queries @ keys.transpose(1, 2)

        # creates bool mask (diagonal) fills all causal invisible elements with -inf
        attn_scores.masked_fill( self.causal_mask.bool()[:con_len, :con_len], -torch.inf)

        # calculations of soft max scaled with sqrt of keys length dimension (keys.shape)
        # note: dim=-1 takes most internal dimension in an array 
        # example: [ [ [ 1, 2, 3] ],  [ [ 4, 5, 6 ], [ 7, 8, 8 ] ] ] -> 3
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        # applying dropout to weights
        # randomly sets weights to 0 based on probability
        # !! scales remaining values up to compensate dropped weights
        attn_weights = self.dropout(attn_weights) # Additional random dropout

        # sum of attention weighted values for every input (including causal masked knowledge)
        context_vec = attn_weights @ values
        return context_vec


## Short insight in masking and causal masking

In [65]:
# Triangular Matrix filtering example for causal masking
def mask_demo():

    # matrix to mask. Example attention scores
    matrix = torch.rand(4,4)

    # buffered static causal_mask
    mask = torch.triu(torch.ones(4, 4), diagonal=1)

    print("\nmatrix (ex. attention_scores):\n", matrix) # some value matrix
    print("\nmask (buffered causal_mask of size context_length):\n", mask) # triangle mask matrix consisting of 0 and 1
    print("\nmask.bool():\n", mask.bool()) # makes matrix with 0 and 1 to true and false
    print("\nmask.bool()[interval] (clipped to match input)\n", mask.bool()[:3, :3]) # sub matrix
    print("\nmatrix.mask_fill(mask.bool()[interval], value)\n", matrix.masked_fill(mask.bool()[:4, :4], -torch.inf))
mask_demo()


matrix (ex. attention_scores):
 tensor([[0.3093, 0.8304, 0.8787, 0.5409],
        [0.9386, 0.2404, 0.3011, 0.7845],
        [0.8350, 0.5887, 0.7426, 0.7358],
        [0.3101, 0.9291, 0.5029, 0.2398]])

mask (buffered causal_mask of size context_length):
 tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])

mask.bool():
 tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])

mask.bool()[interval] (clipped to match input)
 tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

matrix.mask_fill(mask.bool()[interval], value)
 tensor([[0.3093,   -inf,   -inf,   -inf],
        [0.9386, 0.2404,   -inf,   -inf],
        [0.8350, 0.5887, 0.7426,   -inf],
        [0.3101, 0.9291, 0.5029, 0.2398]])


## Multi Head Attention
An implementation of Multi Head Attention
Multiple heads train themself on parts of the input vector. They represent a part of the query, key, value (short qkv) matrices and are combined in the end by linear layer to context vector

In [66]:
class MultiHeadAttention(nn.Module):

    def __init__(self, input_dim, output_dim, context_length, dropout, num_head, qkv_bias=False, verbose=False):
        super().__init__()
        
        # Check if output dimension is dividable by attention head number without rest
        # Input is split to num_head chunks of length head_dim
        assert output_dim % num_head == 0, "Output dimension must be dividable by head_num"

        self.output_dim = output_dim
        self.num_heads = num_head
        self.head_dim = output_dim // num_head

        # Init weight matrices for input to qkv projection (full not yet separated into head parts)
        self.W_query = nn.Linear(input_dim, output_dim, qkv_bias)
        self.W_key = nn.Linear(input_dim, output_dim, qkv_bias)
        self.W_value = nn.Linear(input_dim, output_dim, qkv_bias)

        self.register_buffer("causal_mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))
        
        self.dropout = nn.Dropout(dropout)
        
        self.out_proj = nn.Linear(output_dim, output_dim) # Linear layer for combination of head outputs
        
        if verbose:
            print("input_dim =", input_dim)
            print("output_dim =", output_dim)
            print("num_heads =", num_head)
            print("head_dim =", output_dim)
            print(f"generating nn.Linear({input_dim}, {output_dim}) weights for query, key and value")
            print(f"generating causal diagonal mask torch.triu(torch.ones({context_length}, {context_length}), diagonal=1) for causal masking of attn_scores")        
            print(f"generating dropout nn.Dropout({dropout}) for random dropout of attn_weights")        
            print(f"generating optional nn.Linear({output_dim}, {output_dim}) weights for final context_vector projection")
        
        
    def forward(self, x):
        
        # local variables for input shape
        batch_size, context_length, input_dim = x.shape

        # using weight matrices for projection of the input to qkv (not yet splitted for attention heads)
        # broadcasting for badges
        # -> shape: batch_size, context_length, output_dim
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        # Implicitly splitting matrix by adding head_num dimension
        # Unrol last dim: (batch_size, context_length, output_dim) -> (batch_size, context_length, head_num, head_dim)
        # Example ( 8, 4, 256 ) -> ( 8, 4, 8, 32 ) for num_heads = 8 and head_dim = 32
        queries = queries.view(batch_size, context_length, self.num_heads, self.head_dim)
        keys = keys.view(batch_size, context_length, self.num_heads, self.head_dim)
        values = values.view(batch_size, context_length, self.num_heads, self.head_dim)

        # Transpose to use for query comparison - move num_head to front
        # (batch_size, context_length, num_heads, head_dim) -> (batch_size, num_head, context_length, head_dim)
        queries = queries.transpose(1,2)
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        
        # Compute scaled dot_production attention
        attn_scores = queries @ keys.transpose(2,3) # Dot product for each head

        # Causal Masking
        mask_bool = self.causal_mask.bool()[:context_length, :context_length]
        attn_scores.masked_fill(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim =-1)
        attn_weights = self.dropout(attn_weights)

        # build context vector switch back num_heads and context_length, combining heads
        # self.output_dim = self.num_heads * self.head_dim
        context_vec = (attn_weights @ values).transpose(1,2)
        context_vec = context_vec.contiguous().view(batch_size, context_length, self.output_dim)
        context_vec = self.out_proj(context_vec) # optional projection by Linear layer

        return context_vec

## Test Run

In [71]:
def multi_head_attention_test_run(verbose=False):
    if verbose: print("\n\n------- initializing Multi Head Attention ----------------\n")
    mha = MultiHeadAttention(input_dim=756, output_dim=128, context_length=4, dropout=0.2, num_head=8, verbose=True)

    if verbose: print("\n\n----- Using real data input (see DataPreparation.ipynb) --\n")
    batch = get_test_input_embedding(verbose=verbose) # defined in DataPreparation.ipynb

    if verbose: print("\n\n------- performing multi head attention ------------------\n")
    context_vector = mha(batch)

    if verbose: print("\n Shape of context_vector:\n", context_vector.shape)


_test_run = multi_head_attention_test_run(True)



------- initializing Multi Head Attention ----------------

input_dim = 756
output_dim = 128
num_heads = 8
head_dim = 128
generating nn.Linear(756, 128) weights for query, key and value
generating causal diagonal mask torch.triu(torch.ones(4, 4), diagonal=1) for causal masking of attn_scores
generating dropout nn.Dropout(0.2) for random dropout of attn_weights
generating optional nn.Linear(128, 128) weights for final context_vector projection


----- Using real data input (see DataPreparation.ipynb) --


Init test embedder ...
Generating token_embeddings (50252 x 756)
Generating pos_embeddings (4 x 756)

Displaying first row of batch

First batch elements Input x:
 tensor([   13, 26190,    11, 31174]) . Sebastian, debating

First batch elements Target y:
 tensor([26190,    11, 31174, 12779])  Sebastian, debating possibilities

embeddings[0] for x (4 x 756):
 tensor([[ 1.0139, -0.1002,  2.0504,  ...,  0.8746, -1.4289,  0.5435],
        [-0.1608, -0.5881,  0.7377,  ...,  0.5694,  0.106